In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets
from model_xray.utils.dataset_utils import *

def img_flatten(arr):
    return arr.reshape(arr.shape[0], -1)

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.


In [2]:
def get_train_test_datasets(mc: str, train_x:int, test_xs:Iterable[Union[None, int]] = range(0,24),
                            imsize:int=100, imtype:ImageType=ImageType.GRAYSCALE_FOURPART,
                            flatten:bool=True, normalize:bool=True):
    trainset_name = get_dataset_name(
        mc=mc,
        ds_type='train',
        xs=[train_x, None],
        imsize=imsize,
        imtype=imtype,
    )
    testset_names = {i: get_dataset_name(
        mc=mc,
        ds_type='test',
        xs=[i,],
        imsize=imsize,
        imtype=imtype,
    ) for i in test_xs}
    # print(testset_names)

    ret = retreive_datasets(
        dataset_names=[trainset_name] + list(testset_names.values())
    )

    X_train, y_train = ret[trainset_name]
    if flatten:
        X_train = img_flatten(X_train)

    if normalize:
        X_train = X_train / 255.0

    testsets = {}
    for i, testset_name in testset_names.items():
        X_test, y_test = ret[testset_name]
        if flatten:
            X_test = img_flatten(X_test)

        if normalize:
            X_test = X_test / 255.0

        testsets[i] = (X_test, y_test)

    return ((X_train, y_train), testsets)

mc = 'famous_le_10m'
image_type = ImageType.GRAYSCALE_FOURPART

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

x_train = 22
(X_train, y_train), testsets = get_train_test_datasets(mc, x_train, range(0,24), imtype=image_type, flatten=True, normalize=False)

clf = KNeighborsClassifier(n_neighbors=1, weights='distance',)
clf.fit(X_train, y_train)

train_benign_idxs = np.where(y_train == 0)[0]

X_train_benign = X_train[train_benign_idxs]
y_train_benign = y_train[train_benign_idxs]


print("benign train:", clf.score(X_train_benign, y_train_benign))
print("train:", clf.score(X_train, y_train))

X_test_benigns, y_test_benigns = testsets[0]

print("benign test:", clf.score(X_test_benigns, y_test_benigns))
for i in range(8, 24):
    X_test, y_test = testsets[i]
    print(f"xs{i} test:", clf.score(X_test, y_test))
# print("xs23 test:", rf.score(X_test_xs23_reshaped, y_test_xs23))

benign train: 1.0
train: 1.0
benign test: 0.2
xs8 test: 0.8
xs9 test: 0.8
xs10 test: 0.8
xs11 test: 0.8
xs12 test: 0.8
xs13 test: 0.8
xs14 test: 0.8
xs15 test: 0.8
xs16 test: 0.8
xs17 test: 0.8
xs18 test: 0.8
xs19 test: 0.8
xs20 test: 0.8
xs21 test: 0.8
xs22 test: 1.0
xs23 test: 1.0


# Siamese CNN

In [10]:
from external_code.AI_Model_Steganalysis.siamese import Siamese2

x_train = 22
mc = 'famous_le_10m'
image_type = ImageType.GRAYSCALE_FOURPART

(X_train, y_train), testsets = get_train_test_datasets(mc, x_train, range(0,24), imtype=image_type, flatten=False, normalize=True)

X_test_benigns, y_test_benigns = testsets[0]
X_test_x, y_test_x = testsets[x_train]

model = Siamese2(pretrained=False, img_input_shape=(100,100,1), dist="l2", lr=0.00006)

In [11]:
X_test_benigns.shape

(5, 100, 100)

In [12]:
from itertools import combinations
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from external_code.AI_Model_Steganalysis.siamese import make_triplets, MyThresholdCallback

triplets_train = make_triplets(X_train, y_train)


def get_samples(triplets, idxs):
    anchors = triplets[0][idxs]
    positives = triplets[1][idxs]
    negatives = triplets[2][idxs]

    return [anchors, positives, negatives]

model.fit(triplets_train, epochs=5, batch_size=16, callbacks=[])

Epoch 1/5


2/2 [==============================] - 1s 10ms/step - loss: 0.9203
Epoch 2/5
2/2 [==============================] - 0s 9ms/step - loss: 2.2169
Epoch 3/5
2/2 [==============================] - 0s 9ms/step - loss: 2.4914
Epoch 4/5
2/2 [==============================] - 0s 10ms/step - loss: 0.7928
Epoch 5/5
2/2 [==============================] - 0s 9ms/step - loss: 0.1860


In [13]:
print("Train Centroid:")
ret = model.test_centroid(X_train, y_train, X_train, y_train, is_print=True, apply_transforms=False, knn=True, vanilla=False)

print("\nTrain KNN:")
ret = model.test_nn(X_train, y_train, X_train, y_train, is_print=True)

Train Centroid:
knn (centroid) accuracy: 0.6666666666666666

Train KNN:
accuracy: 1.0


In [14]:
X_test = np.concatenate([X_test_benigns])
y_test = np.concatenate([y_test_benigns])

print("\nTest Centroid (benigns):")
ret = model.test_centroid(X_train, y_train, X_test, y_test, is_print=True, apply_transforms=False, knn=True, vanilla=False)

print("\nTest KNN (benigns):")
ret = model.test_nn(X_train, y_train, X_test, y_test, is_print=True)

X_test = np.concatenate([X_test_x])
y_test = np.concatenate([y_test_x])

print("\nTest Centroid (train_x):")
ret = model.test_centroid(X_train, y_train, X_test, y_test, is_print=True, apply_transforms=False, knn=True, vanilla=False)

print("\nTest KNN (train_x):")
ret = model.test_nn(X_train, y_train, X_test, y_test, is_print=True)


Test Centroid (benigns):
knn (centroid) accuracy: 0.2

Test KNN (benigns):
accuracy: 0.4

Test Centroid (train_x):
knn (centroid) accuracy: 1.0

Test KNN (train_x):
accuracy: 0.8


# All CNNs, All Attacks

In [6]:
dataset_name = f"allcnns_allattacks_train_imsize256"

ret = retreive_datasets(
    dataset_names=[dataset_name]
)

X,y = ret[dataset_name]

In [8]:
print(np.unique(y, return_counts=True))
X.shape

(array([0, 1]), array([ 24, 552]))


(576, 256, 256)

In [14]:
X = img_flatten(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42, stratify=y)

In [29]:
X_train = X[y == 0]
y_train = y[y == 0]
X_test = X[y == 1]
y_test = y[y == 1]

In [30]:
X_train.shape

(24, 65536)

In [31]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [45]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor

from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM

# clf = KNeighborsClassifier(n_neighbors=11, weights='distance',)
# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

# X_train_benign = X_train[y_train == 0]

# clf = IsolationForest(max_samples=100)
# clf = OneClassSVM()
clf = LocalOutlierFactor(n_neighbors=7, novelty=True,)
clf.fit(X_train,)

def map_outliers(y):
    return [0 if i == 1 else 1 for i in y]

y_pred_train = map_outliers(clf.predict(X_train))

print("train: ")
print(classification_report(y_train, y_pred_train))

train: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        24
           1       0.00      0.00      0.00         0

    accuracy                           0.92        24
   macro avg       0.50      0.46      0.48        24
weighted avg       1.00      0.92      0.96        24



/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [46]:
y_pred_test = map_outliers(clf.predict(X_test))

print("test: ")
print(classification_report(y_test, y_pred_test))

test: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.08      0.15       552

    accuracy                           0.08       552
   macro avg       0.50      0.04      0.07       552
weighted avg       1.00      0.08      0.15       552



/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

# Siamese CNN